In [13]:
import dolphindb as ddb
import time
import pandas as pd

In [14]:
n = 10

# 读取dfs数据（仅读取数据）

In [31]:
total_time = 0
num_runs = 1000

# 连接服务器节点
s = ddb.session()
isConnect = s.connect("115.239.209.123", 8892, "admin", "123456")  # 连接数据库
print(isConnect)

for _ in range(num_runs):
    start = time.perf_counter()
    
    ## 你的代码段
    script2 = """
    // STATIC数据
    STATIC = loadTable("dfs://A_STATIC",`pt)

    // GUOKAI数据
    FILLED_GUOKAI = loadTable('dfs://A_GUOKAI',`pt1)

    // 读取BONDS数据
    BONDS = loadTable("dfs://A_BONDS",`pt2)
    """
    s.run(script2)
    
    end = time.perf_counter()
    runTime = end - start
    total_time += runTime

# 计算平均时间
avg_run_time = total_time / num_runs
avg_run_time_ms = avg_run_time * 1000

# 输出平均运行时间
print(f"平均运行时间：{avg_run_time} 秒")
print(f"平均运行时间：{avg_run_time_ms} 毫秒")


True
平均运行时间：0.05736820169976272 秒
平均运行时间：57.36820169976272 毫秒


# 读取dfs数据（读取+处理日期）

__注：  
存储的时候因为把trade_date作为分区列，将其从string vector转成了date vector，因此在读取的时候把它转变回来。  
但是我的写法可能效率不高，说不定可以改进__

In [16]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [17]:
# 连接服务器节点
s = ddb.session()
isConnect = s.connect("115.239.209.123", 8892, "admin", "123456")  # 连接数据库
print(isConnect)

True


In [18]:
# 读取表 + 处理日期数据

script = """
// STATIC数据
STATIC = loadTable("dfs://A_STATIC",`pt)

// GUOKAI数据
// 读取表  注意和原表不同了，需要把TRADE_DT从日期变回string
FILLED_GUOKAI = loadTable('dfs://A_GUOKAI',`pt1)

FILLED_GUOKAI = select * , strReplace(string(TRADE_DT), ".", "") from FILLED_GUOKAI
FILLED_GUOKAI.dropColumns!(`TRADE_DT)
FILLED_GUOKAI.rename!(`strReplace, `TRADE_DT)


// 读取BONDS数据
BONDS = loadTable("dfs://A_BONDS",`pt2)

BONDS = select * , strReplace(string(TRADE_DT), ".", "") from BONDS
BONDS.dropColumns!(`TRADE_DT)
BONDS.rename!(`strReplace, `TRADE_DT)
"""

for i in range(10):
    s.run(script)

In [19]:
# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime = runTime/n
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")

运行时间： 2.3553763599993545 秒
运行时间： 2355.3763599993545 毫秒


# 读取本地parquet文件

In [20]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [21]:
for i in range(n):
    # 读取本地数据
    STATIC = pd.read_parquet('./Results/STATIC.parquet')     # 初始行数21930
    BONDS = pd.read_parquet('./Results/BONDS.parquet')
    FILLED_GUOKAI = pd.read_parquet('./Results/FILLED_GUOKAI.parquet')

In [22]:
# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime = runTime/n
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")

运行时间： 2.5964221799993537 秒
运行时间： 2596.422179999354 毫秒


# 读取服务器端parquet文件

In [23]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [24]:
# 连接服务器节点
s = ddb.session()
isConnect = s.connect("192.168.100.43", 7988, "admin", "123456")  # 连接数据库
print(isConnect)

script = """

STATIC = parquet::loadParquet("/home/hanyang/luodata/STATIC.parquet")

BONDS = parquet::loadParquet("/home/hanyang/luodata/BONDS.parquet")

FILLED_GUOKAI = parquet::loadParquet("/home/hanyang/luodata/FILLED_GUOKAI.parquet")

"""

s.run(script)

for i in range(n):
    s.run(script)

True


In [25]:
# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime = runTime/n
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")


运行时间： 19.19155965999962 秒
运行时间： 19191.55965999962 毫秒


# 结果汇总（平均结果）

| 运行时间   | 读取dfs数据（仅读表） | 读取dfs数据（读表+处理日期） | 读取本地parquet数据 | 读取服务器端parquet数据 |
| --------- | --------------------- | -------------------------- | -------------------- | ---------------------- |
| 秒        | 0.0573             | 2.3553                     | 2.5964                 | 19.1915                  |
| 毫秒      | 57.3682                 |  2355.3763                  | 2596.4221              | 19191.5596               |


__说明：__

读取dfs数据（仅读表）循环10次结果仍然不稳定，因此设置运行了1000次取平均。

其他读取结果为循环运行10次取平均的结果，基本稳定，多次运行数值不会改变